# Tutorial 1: Fetching and Visualizing WorldClim Data

This notebook demonstrates how to programmatically fetch climate data from WorldClim, specifically looking at minimum temperature data.

**Objective:** Download a GeoTIFF file, load it using `rasterio`, and visualize the spatial data.

**Themes covered:**
- GeoTIFF format handling
- Spatial data visualization
- Programmatic data retrieval

In [ ]:
import os
import requests
import rasterio
import matplotlib.pyplot as plt
from rasterio.plot import show
import numpy as np

# Ensure the data directory exists
os.makedirs("data", exist_ok=True)

# 1. Define the WorldClim URL for minimum temperature (10m resolution)
# We use the 10 minute resolution for a smaller file size in this demo
url = "https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_10m_tmin.zip"
zip_path = "data/wc2.1_10m_tmin.zip"

print(f"Downloading data from {url}...")
print("This might take a while depending on your internet connection.")

# 2. Download the file
if not os.path.exists(zip_path):
    try:
        response = requests.get(url, stream=True, timeout=30)
        response.raise_for_status()
        with open(zip_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print("Download complete.")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading: {e}")
        print("\n*** MANUAL DOWNLOAD INSTRUCTIONS ***")
        print(f"1. Open this URL in your browser: {url}")
        print(f"2. Save the file to: {os.path.abspath(zip_path)}")
        print("3. Run the next cells once the file is present.")
else:
    print("File already exists.")


## Extracting and Loading the Data

The data comes in a ZIP file containing GeoTIFFs for each month. We will extract one month (January) to visualize.

In [ ]:
import zipfile

# Extract specific file (January Minimum Temperature)
target_file = "wc2.1_10m_tmin_01.tif"
extracted_path = os.path.join("data", target_file)

if os.path.exists(zip_path) and not os.path.exists(extracted_path):
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extract(target_file, "data")
            print(f"Extracted {target_file}")
    except zipfile.BadZipFile:
        print("Error: The ZIP file seems corrupted. Please delete it and try downloading again.")
elif os.path.exists(extracted_path):
    print(f"{target_file} already extracted.")
else:
    print("ZIP file not found. Please ensure it is downloaded.")

## Visualizing the GeoTIFF

We use `rasterio` to open the file and `matplotlib` to plot it.

In [ ]:
if os.path.exists(extracted_path):
    with rasterio.open(extracted_path) as src:
        print("Metadata:", src.profile)
        
        # Read the data (band 1)
        data = src.read(1)
        
        # Mask no-data values for better visualization
        data = np.where(data == src.nodata, np.nan, data)

        plt.figure(figsize=(12, 8))
        plt.title("Global Minimum Temperature - January (WorldClim)")
        plt.imshow(data, cmap='coolwarm', interpolation='none')
        plt.colorbar(label='Temperature (°C)')
        plt.show()
else:
    print("Data file not found. Ensure download and extraction were successful.")